# Usage of UWB prototype

This notebook shows the general procedure for using the different moduls. Note that most components do not
satisfy the performance contraints for most practical applications. The general id

In [ ]:
# first we import the required modules and set up some inital particles and weights
import numpy as np

from uwb.generator import BlobGenerator
from uwb.map import NoiseMapGM
from uwb.algorithm import MNMAParticleFilter

init_particles = np.stack([np.arange(10), (np.arange(10) + 1) ** 2, (np.arange(10) + 5) ** 3]).T
init_weights = np.ones(10) * 0.1

### Noise map generation

The first step is to construct a noise map from location measurements. For this notebook, we will synthesize the data for a simple grid space with clean Gaussian mixtures for each position.

In [ ]:
np.random.seed(101)

generator = BlobGenerator(
        grid_dims=[4, 4, 4], # this is a grid for a 4x4x4 cube
        step_size=10, # distance between measurement locations
        measurements_per_location=50,
        modal_range=(1,3), # how many mixture components should be used
        deviation=2.0,
        )

noise_map = NoiseMapGM(generator=generator)
noise_map.gen()  # calculation of all parameters for the noise map

weights, means, covs = noise_map[0, 0, 0]  # example gaussian mixture parameters for location 10x10x10
print("weights: ", weights, "\n\n means: ", means, "\n\n covariances", covs)

### Particle Filter

The next step is to construct the noise map augmented particle filter (basic particle filter is also provided
in this package).

In [ ]:
pf = MNMAParticleFilter(
    init_particles,
    init_weights,
    map=noise_map,
)

In [ ]:
# now we can resample
pf.resample()
pf.particles

In [ ]:
# if we are provided measurements we can also update weights with

measurements = np.array([[25.0, 25.0, 25.0], [30.0, 30.0, 30.0]])  # note first dimension is the batch size
pf.update_weights(measurements)

pf.weights

In [ ]:
pf.resample()
pf.weights